In [1]:
def get_responder_type(res_file_path):
    responsders_type = ["EMT-Basic","AEMT","First Responder","Paramedic"]
    responders = res_file_path.split("/")
    for res_type in responsders_type:
        if res_type in responders:
            return res_type

    return None

def get_state(state_path):
    states = state_path.split("/")

    for state in states:

        if len(state)==2:
            return state

    return None

In [13]:
import os
from src.utils.pdf_parsers import PdfParser
from qdrant_client import QdrantClient
import uuid


# folders = ["General Information","Specific Files","Combo Files/Local Combo","Combo Files/State Combo","Specific Files"]
folders = ["Specific Files"]
root_dir = "KB"

db = []


client = QdrantClient(path="./Kb_db/")  # Persists changes to disk

for fold in folders:

    for root, dirs, files in os.walk(os.path.join(root_dir,fold)):
        for file in files:
            local_file_path = os.path.join(root,file)
            if local_file_path.endswith('.pdf'):
                parser = PdfParser(local_file_path)
                paragraphs = parser.parse()

                metadata = [{"type":get_responder_type(local_file_path),"state":get_state(local_file_path)
                             } for i in range(len(paragraphs))]
                vector_ids = [str(uuid.uuid4()) for _ in range(len(paragraphs))]
                
                client.add(
                    collection_name=fold,
                    documents=paragraphs,
                    metadata=metadata,
                    ids=vector_ids
                )
                # db.append(
                #     {
                #         "corpus":paragraphs,
                #         "metadata":metadata 
                #     }
                # )

                del paragraphs
                del metadata



RuntimeError: Storage folder ./Kb_db/ is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

In [11]:
str(uuid.uuid4())

'10afb76e-dfef-482c-a36b-4ef7bb1aa9c9'

In [ ]:
# Create Vectors
# Create Metadata
# Create Collection
# Upsert Data

In [26]:
historical_facts = [
    "Egypt: The construction of the Great Pyramid of Giza, one of the Seven Wonders of the Ancient World, was completed around 2560 BCE.",
    "China: The Great Wall of China, a UNESCO World Heritage Site, was built over several dynasties and stretches over 13,000 miles.",
    "Greece: The ancient Olympic Games, held in Olympia, began in 776 BCE and continued for over a millennium until they were abolished in 393 CE.",
    "Rome: The Roman Empire, at its height in the 2nd century AD, was the largest empire in ancient history, spanning three continents.",
    "India: The Maurya Empire, led by Emperor Ashoka, flourished in the 3rd century BCE and played a key role in the spread of Buddhism.",
    "Mesopotamia: The Code of Hammurabi, one of the earliest and most complete written legal codes, dates back to around 1754 BCE in Babylon.",
    "England: The Magna Carta, signed in 1215, laid the foundation for constitutional governance and influenced the development of modern legal systems.",
    "Japan: The Edo period (1603-1868) marked a time of peace and isolation, with the samurai class playing a central role in Japanese society.",
    "Russia: The Russian Revolution of 1917 led to the establishment of the Soviet Union, dramatically altering the course of 20th-century history.",
    "United States: The Declaration of Independence, adopted on July 4, 1776, proclaimed the thirteen American colonies as independent states from British rule."
]


# meta data

metadata = [{"country":fact.split(":")[0]} for fact in historical_facts]
metadata

[{'country': 'Egypt'},
 {'country': 'China'},
 {'country': 'Greece'},
 {'country': 'Rome'},
 {'country': 'India'},
 {'country': 'Mesopotamia'},
 {'country': 'England'},
 {'country': 'Japan'},
 {'country': 'Russia'},
 {'country': 'United States'}]

In [27]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(api_key="")

In [28]:
result = embedding.embed_documents(historical_facts)

In [60]:
from qdrant_client.models import Distance, VectorParams, PointStruct
from qdrant_client import QdrantClient


client = QdrantClient(path="./KB-Vector-Store/")

# client.create_collection(collection_name="test2",
#                          vectors_config=VectorParams(size=1536,distance=Distance.COSINE))

In [61]:
client.get_collections().collections

[CollectionDescription(name='Specific Files')]

In [62]:
emb_model = OpenAIEmbeddings(api_key="")


In [63]:
vector = emb_model.embed_query("Medical related stuff")
res = client.search(
        collection_name="Specific Files",
        query_vector=vector,
        limit=10,
)

In [67]:
res[0].payload['text']

'a nd cleaning and disposing of equipment and supplies a principles of body substance isolation b hand washing guidelines c recommendations for ppe d recommendations for cleaning or sterilization of equipment e recommendations for disposing of cont aminated linens and supplies including sharps f recommendations for decontaminating the ambulance iii consider agerelated variations in pediat ric and geriatric patients as they relate assessment and management of patients w ith a gastrointestinal condition or emergency iv communication and documentation for a patie nt with a communicable or infectious disease v transport decisions including special infection control procedures page 92 of 212 vi legal requirements regarding repor ting communicable or infectious diseasesconditions a exposure of health care provider 1 current recommended treatment modalities and followup 2 prevention of exposure or immunizationsvaccines vii required reporting to the h ealth department or other health care agen

In [43]:
import uuid
points = []

for fact, meta in zip(historical_facts,metadata):
    vector = embedding.embed_query(fact)
    meta['text'] = fact
    points.append(PointStruct(id=str(uuid.uuid4()),payload=meta,vector=vector))
    


In [44]:
points

[PointStruct(id='65b26b89-fb4a-4dcb-8ca8-adf3051d0260', vector=[0.01417042815565645, -0.016094807931783445, 0.0066041193666271265, -0.031639792804295575, -0.012645920118691593, -0.003078694298315242, -0.021930425036621168, -0.003879998207651709, -0.007103957876426548, -0.008634714622359678, 0.025579248253632787, 0.023092549862943134, 0.00296623057540271, -0.008128627403591984, -0.005292042754534684, 0.012183568740427517, 0.0073226375446059375, -0.0021040086221297476, -0.006504151664667242, 0.01435786800428487, -0.011252619603237589, -0.02275515776288294, -0.002455457872646735, -0.01492018661884753, 0.006282348340495665, 0.005576325717808203, -0.0073726211627552305, -0.016482182253009035, 0.02671638010672686, -0.0028209649721124643, -0.0035488550493904367, -0.01895638415708474, -0.026866332358158634, -0.005613813780666147, -0.045010478412950354, -0.006829046812452191, -0.007503829149927383, -0.027841017335852182, 0.04356094557037878, 0.019056352324705924, 0.02131812140692532, 0.02289261

In [45]:
client.upsert(collection_name="test2",points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [35]:
fact = "Tell me about China"
vect = embedding.embed_query(fact)
hits = client.search(
    collection_name="test",
    query_vector=vect,

)

In [36]:
hits

[ScoredPoint(id='5de90281-43ac-439f-859b-9c235726ed69', version=0, score=0.8249436354035007, payload={'country': 'China', 'text': 'China: The Great Wall of China, a UNESCO World Heritage Site, was built over several dynasties and stretches over 13,000 miles.'}, vector=None, shard_key=None),
 ScoredPoint(id='6eaf4e81-db40-4cc2-9447-ea659b81a765', version=0, score=0.7649433766034197, payload={'country': 'Rome', 'text': 'Rome: The Roman Empire, at its height in the 2nd century AD, was the largest empire in ancient history, spanning three continents.'}, vector=None, shard_key=None),
 ScoredPoint(id='eb961529-ab2b-4301-b72c-0f144f932ea2', version=0, score=0.7629905609886132, payload={'country': 'India', 'text': 'India: The Maurya Empire, led by Emperor Ashoka, flourished in the 3rd century BCE and played a key role in the spread of Buddhism.'}, vector=None, shard_key=None),
 ScoredPoint(id='60c5e305-a735-43c1-9814-a62efca071b7', version=0, score=0.7612019546866702, payload={'country': 'Japa

In [51]:
client.get_collections().collections[0].name

'test'

In [55]:
collection_names = [collection.name for collection in client.get_collections().collections]

In [56]:
collection_names

['test', 'test2']

In [1]:
a = []

In [2]:
a.extend([12,3])
a

[12, 3]

In [2]:
def recursive(prompt,index,breaking_cond):
    if(index>breaking_cond):
        return 
    
    print(prompt)
    print(index)
    recursive(prompt,index+1,breaking_cond)
    
    
prompt = "Hello"
recursive(prompt,0,5)
    

Hello
0
Hello
1
Hello
2
Hello
3
Hello
4
Hello
5


In [9]:
folders = ['Combo Files/State Combo']
# folders = ['sp']

In [10]:
'-'.join(folders[0].split("/"))

'Combo Files-State Combo'

In [ ]:
client = QdrantClient(path="./KB-Vector-Store/")  # Persists changes to disk
     